In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]
RESULTS_DIR = 'results_003_lang_id_classification_2_weight_decay'
GPU_ID = 0

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
bias_category_specification = [
    {
        "category": "utterance",
        "subcategories": [
            {
                "subcategory": "verbal_nod",
                "tags": set(["utt-verbal-nod"])
            },
            {
                "subcategory": "multilingual",
                "tags": set(["utt-multi-lingual", "utt-multi-lingual-named-endity"])
            }
        ]
    },
    {
        "category": "speaker_count",
        "subcategories": [
            {
                "subcategory": "single",
                "tags": set(["spkr-single"])
            },
            {
                "subcategory": "multiple",
                "tags": set(["spkr-mult", "spkr-multi"])
            }
        ]
    },
    {
        "category": "gender",
        "subcategories": [
            {
                "subcategory": "male",
                "tags": set(["spkr-male"])
            },
            {
                "subcategory": "female",
                "tags": set(["spkr-female"])
            },
        ]
    },
    {
        "category": "language",
        "subcategories": [
            {
                "subcategory": "susu",
                "tags": set(["lng-susu"])
            },
            {
                "subcategory": "maninka",
                "tags": set(["lng-maninka"])
            },
            {
                "subcategory": "pular",
                "tags": set(["lng-pular"])
            }
        ]
    },
    {
        "category": "channel",
        "subcategories": [
            {
                "subcategory": "telephone",
                "tags": set(["ct-telephone"])
            },
            {
                "subcategory": "noise",
                "tags": set(["ct-noise"])
            },
            {
                "subcategory": "music",
                "tags": set(["ct-fg-music", "ct-tr-music", "ct-bg-music"])
            }
        ]
    }
]

flat_bias_category_specification = {}

for c in bias_category_specification:
    for sc in c['subcategories']:
        k = f"{c['category']}_{sc['subcategory']}"
        flat_bias_category_specification[k] = sc['tags']

_ = [print(f"{k}: {v}") for k,v in flat_bias_category_specification.items()]

utterance_verbal_nod: {'utt-verbal-nod'}
utterance_multilingual: {'utt-multi-lingual-named-endity', 'utt-multi-lingual'}
speaker_count_single: {'spkr-single'}
speaker_count_multiple: {'spkr-multi', 'spkr-mult'}
gender_male: {'spkr-male'}
gender_female: {'spkr-female'}
language_susu: {'lng-susu'}
language_maninka: {'lng-maninka'}
language_pular: {'lng-pular'}
channel_telephone: {'ct-telephone'}
channel_noise: {'ct-noise'}
channel_music: {'ct-bg-music', 'ct-fg-music', 'ct-tr-music'}


In [5]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [6]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label, tag_set
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels, audio_tags = zip(*data)

## Inspect label counts

In [7]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [8]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels, audio_tags = zip(*balanced_data)

In [9]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


## Inspect bias category counts in balanded data

In [10]:
def inspect_bias_category_counts():
    for name, tags in flat_bias_category_specification.items():
        count = len([ts for ts in audio_tags if len(tags.intersection(ts))>0])
        print(name, count, "/", len(audio_tags))
            
inspect_bias_category_counts()

utterance_verbal_nod 48 / 84
utterance_multilingual 21 / 84
speaker_count_single 25 / 84
speaker_count_multiple 58 / 84
gender_male 81 / 84
gender_female 15 / 84
language_susu 28 / 84
language_maninka 28 / 84
language_pular 28 / 84
channel_telephone 27 / 84
channel_noise 21 / 84
channel_music 24 / 84


# Prepare 10 cross validation folds

In [11]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(test_index_set)),
    }


# Load features

In [12]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [13]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [14]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features[feature_name][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [15]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [16]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [17]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [19]:
class LangIdLogisticReg1(nn.Module):
    def __init__(self):
        super(LangIdLogisticReg1, self).__init__()
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        x = self.lin1(x)
        return x
    
    
class LangIdLogisticReg2(nn.Module):
    def __init__(self):
        super(LangIdLogisticReg2, self).__init__()
        self.drop0 = nn.Dropout(p=0.5)
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        x = self.drop0(x)
        x = self.lin1(x)
        return x
    
    
class LangIdDNN1(nn.Module):
    def __init__(self):
        super(LangIdDNN1, self).__init__()
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        self.drop1 = nn.Dropout(p=0.3)
        
        self.lin2 = nn.Linear(in_features=3, out_features=8)
        self.drop2 = nn.Dropout(p=0.3)
        
        self.lin3 = nn.Linear(in_features=8, out_features=3)
        
        
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        
        x = self.lin1(x)
        x = F.elu(x)
        x = self.drop1(x)
        
        x = self.lin2(x)
        x = F.elu(x)
        x = self.drop2(x)
        
        x = self.lin3(x)
        
        return x
    
    
    
    
class LangIdDNN2(nn.Module):
    def __init__(self):
        super(LangIdDNN2, self).__init__()
        self.drop0 = nn.Dropout(p=0.5)
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        self.drop1 = nn.Dropout(p=0.3)
        
        self.lin2 = nn.Linear(in_features=3, out_features=8)
        self.drop2 = nn.Dropout(p=0.3)
        
        self.lin3 = nn.Linear(in_features=8, out_features=3)
        
        
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        
        x = self.drop0(x)
        
        x = self.lin1(x)
        x = F.elu(x)
        x = self.drop1(x)
        
        x = self.lin2(x)
        x = F.elu(x)
        x = self.drop2(x)
        
        x = self.lin3(x)
        
        return x
    
    
    
class LangIdCNN1(nn.Module):
    def __init__(self):
        super(LangIdCNN1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=0.3)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=0.3)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.pool4(x)
        
        x = torch.mean(x, dim=2)
        
        return x

    
class LangIdCNN2(nn.Module):
    def __init__(self):
        super(LangIdCNN2, self).__init__()
        self.drop0 = nn.Dropout(p=0.5)
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=0.3)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=0.3)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.drop0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.pool4(x)
        
        x = torch.mean(x, dim=2)
        
        return x
    
    
class LangIdRNN1(nn.Module):
    def __init__(self):
        super(LangIdRNN1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.gru = nn.GRU(input_size=1, hidden_size=3, num_layers=1, batch_first=True)
        #self.drop2 = nn.Dropout(p=0.3)
        self.lin2 = nn.Linear(in_features=3, out_features=3)
        
        
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = x.permute(0, 2, 1)
        # output, (h_n, c_n) = self.lstm(x)
        output, h_n = self.gru(x)
        x = output[:, -1, :] # [all batches, last time step, all neurons]
        x = self.lin2(x)
        
        return x
    
    
class LangIdRNN2(nn.Module):
    def __init__(self):
        super(LangIdRNN2, self).__init__()
        self.drop0 = nn.Dropout(p=0.5)
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.gru = nn.GRU(input_size=1, hidden_size=3, num_layers=1, batch_first=True)
        #self.drop2 = nn.Dropout(p=0.3)
        self.lin2 = nn.Linear(in_features=3, out_features=3)
        
        
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.drop0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = x.permute(0, 2, 1)
        # output, (h_n, c_n) = self.lstm(x)
        output, h_n = self.gru(x)
        x = output[:, -1, :] # [all batches, last time step, all neurons]
        x = self.lin2(x)
        
        return x


# Train Classification Models

In [20]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']    

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)
    train_tags = np.take(audio_tags, train_indices, axis=0)
    train_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        train_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in train_tags]

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    test_tags = np.take(audio_tags, test_indices, axis=0)
    test_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        test_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in test_tags]
        
    
    return train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels = \
        get_data_for_fold(fold_id, feature_name, batch_size)
    
    
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, train_bias_category_labels, test_bias_category_labels


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [21]:
def train_on_fold(model_class, fold_id, feature_name, batch_size, epochs):
    device = torch.device(f"cuda:{GPU_ID}")
    torch.manual_seed(0)
    results = {}
    
    model = model_class().to(device)

    train_loader, test_loader, train_bias_category_labels, test_bias_category_labels = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(model_class(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )
            true_train_classes.extend(y.cpu().numpy())
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_n = len(true_train_classes)
        
        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)
        train_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes, sample_weight=sw)
            for category, sw in train_bias_category_labels.items()
        }
        


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )

            true_test_classes.extend(y.cpu().numpy())

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_n = len(true_test_classes)
        
        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)
        test_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes, sample_weight=sw)
            for category, sw in test_bias_category_labels.items()
        }

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'epoch': epoch,
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_n': train_n,
            'test_n': test_n,
            
        }
        
        for c in train_acc_by_bais_category:
            results[epoch][f"train_acc_{c}"] = train_acc_by_bais_category[c]
            results[epoch][f"train_n_{c}"] = int(np.sum(train_bias_category_labels[c]))
            
        for c in test_acc_by_bais_category:
            results[epoch][f"test_acc_{c}"] = test_acc_by_bais_category[c]
            results[epoch][f"test_n_{c}"] = int(np.sum(test_bias_category_labels[c]))
        
    del model
    return results
    

In [22]:
import csv
from pathlib import Path

def save_results(model_name, all_results):
    for result_entry in all_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        
        Path(RESULTS_DIR).mkdir(exist_ok=True)
        fname = f"{RESULTS_DIR}/{model_name}/{feature_name}_{fold_index}.csv"
        Path(fname).parent.mkdir(parents=True, exist_ok=True)
        with open(fname, 'w') as f:
            fieldnames = sorted(result_entry['epochs'][1].keys())
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for epoch in sorted(result_entry['epochs'].keys()):

                writer.writerow(result_entry['epochs'][epoch])

In [23]:
model_classes = [
    LangIdLogisticReg1, 
    LangIdLogisticReg2, 
    LangIdDNN1, 
    LangIdDNN2, 
    LangIdCNN1, 
    LangIdCNN2, 
    LangIdRNN1,
    LangIdRNN2
]

all_results = []

for fold_index in cv_folds:
    for model_class in model_classes:
        for feature_name in raw_features:
            print(f"{model_class.__name__} using {feature_name} on fold#{fold_index}")
            resutls = train_on_fold(model_class, fold_index, feature_name, batch_size=100, epochs=1000)
            all_results.append({
                'fold_index': fold_index,
                'feature_name': feature_name,
                'epochs': resutls
            })
            save_results(model_class.__name__, all_results)

LangIdLogisticReg1 using wav2vec_features-c on fold#0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Linear-1             [10, 3]           1,539           1,539
Total params: 1,539
Trainable params: 1,539
Non-trainable params: 0
-----------------------------------------------------------------------
Epoch: 10. Train Loss: 1.073. Test Loss: 1.085. Train Acc: 0.5294. Test Acc:0.4242
Epoch: 20. Train Loss: 1.047. Test Loss: 1.07. Train Acc: 0.7059. Test Acc:0.5152
Epoch: 30. Train Loss: 1.024. Test Loss: 1.058. Train Acc: 0.6863. Test Acc:0.5455
Epoch: 40. Train Loss: 1.001. Test Loss: 1.046. Train Acc: 0.7255. Test Acc:0.5758
Epoch: 50. Train Loss: 0.98. Test Loss: 1.037. Train Acc: 0.7451. Test Acc:0.5758
Epoch: 60. Train Loss: 0.9601. Test Loss: 1.028. Train Acc: 0.7647. Test Acc:0.6061
Epoch: 70. Train Loss: 0.9415. Test Loss: 1.021. Train Acc: 0.7647. Test Acc:0.6061
Epoch: 80. 

Epoch: 920. Train Loss: 0.4626. Test Loss: 1.058. Train Acc: 0.8824. Test Acc:0.5152
Epoch: 930. Train Loss: 0.4608. Test Loss: 1.059. Train Acc: 0.902. Test Acc:0.5152
Epoch: 940. Train Loss: 0.4591. Test Loss: 1.06. Train Acc: 0.902. Test Acc:0.5152
Epoch: 950. Train Loss: 0.4573. Test Loss: 1.06. Train Acc: 0.902. Test Acc:0.5152
Epoch: 960. Train Loss: 0.4556. Test Loss: 1.061. Train Acc: 0.902. Test Acc:0.5152
Epoch: 970. Train Loss: 0.4539. Test Loss: 1.062. Train Acc: 0.902. Test Acc:0.5152
Epoch: 980. Train Loss: 0.4523. Test Loss: 1.062. Train Acc: 0.902. Test Acc:0.5152
Epoch: 990. Train Loss: 0.4507. Test Loss: 1.063. Train Acc: 0.902. Test Acc:0.5152
Epoch: 1000. Train Loss: 0.4491. Test Loss: 1.064. Train Acc: 0.902. Test Acc:0.5152
LangIdLogisticReg1 using wav2vec_features-z on fold#0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Linear-1             [10, 3]         

Epoch: 830. Train Loss: 0.8069. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 840. Train Loss: 0.8057. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 850. Train Loss: 0.8046. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 860. Train Loss: 0.8035. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 870. Train Loss: 0.8024. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 880. Train Loss: 0.8014. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 890. Train Loss: 0.8003. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 900. Train Loss: 0.7993. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 910. Train Loss: 0.7983. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 920. Train Loss: 0.7973. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 930. Train Loss: 0.7963. Test Loss: 1.059. Train Acc: 0.8039. Test Acc:0.4545
Epoch: 940. Train Loss: 0.7954. Test Loss: 1.059. Train Acc: 0.80

Epoch: 740. Train Loss: 0.7166. Test Loss: 0.9823. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 750. Train Loss: 0.7147. Test Loss: 0.982. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 760. Train Loss: 0.7129. Test Loss: 0.9818. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 770. Train Loss: 0.711. Test Loss: 0.9815. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 780. Train Loss: 0.7093. Test Loss: 0.9813. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 790. Train Loss: 0.7075. Test Loss: 0.9811. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 800. Train Loss: 0.7058. Test Loss: 0.9809. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 810. Train Loss: 0.7041. Test Loss: 0.9807. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 820. Train Loss: 0.7024. Test Loss: 0.9805. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 830. Train Loss: 0.7008. Test Loss: 0.9803. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 840. Train Loss: 0.6992. Test Loss: 0.9801. Train Acc: 0.9216. Test Acc:0.4848
Epoch: 850. Train Loss: 0.6976. Test Loss: 0.9799. Train

Epoch: 640. Train Loss: 0.664. Test Loss: 0.9935. Train Acc: 0.8431. Test Acc:0.5152
Epoch: 650. Train Loss: 0.661. Test Loss: 0.9932. Train Acc: 0.8431. Test Acc:0.5152
Epoch: 660. Train Loss: 0.6582. Test Loss: 0.9928. Train Acc: 0.8431. Test Acc:0.5152
Epoch: 670. Train Loss: 0.6554. Test Loss: 0.9925. Train Acc: 0.8431. Test Acc:0.5152
Epoch: 680. Train Loss: 0.6526. Test Loss: 0.9922. Train Acc: 0.8431. Test Acc:0.5152
Epoch: 690. Train Loss: 0.6499. Test Loss: 0.9919. Train Acc: 0.8431. Test Acc:0.5455
Epoch: 700. Train Loss: 0.6473. Test Loss: 0.9916. Train Acc: 0.8627. Test Acc:0.5455
Epoch: 710. Train Loss: 0.6447. Test Loss: 0.9914. Train Acc: 0.8627. Test Acc:0.5455
Epoch: 720. Train Loss: 0.6421. Test Loss: 0.9911. Train Acc: 0.8824. Test Acc:0.5455
Epoch: 730. Train Loss: 0.6396. Test Loss: 0.9908. Train Acc: 0.8824. Test Acc:0.5455
Epoch: 740. Train Loss: 0.6371. Test Loss: 0.9906. Train Acc: 0.8824. Test Acc:0.5455
Epoch: 750. Train Loss: 0.6347. Test Loss: 0.9904. Train

...

Epoch: 20. Train Loss: 1.132. Test Loss: 1.126. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 30. Train Loss: 1.116. Test Loss: 1.12. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 40. Train Loss: 1.103. Test Loss: 1.115. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 50. Train Loss: 1.093. Test Loss: 1.111. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 60. Train Loss: 1.08. Test Loss: 1.108. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 70. Train Loss: 1.064. Test Loss: 1.105. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 80. Train Loss: 1.057. Test Loss: 1.104. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 90. Train Loss: 1.041. Test Loss: 1.103. Train Acc: 0.3725. Test Acc:0.3333
Epoch: 100. Train Loss: 1.031. Test Loss: 1.103. Train Acc: 0.3725. Test Acc:0.303
Epoch: 110. Train Loss: 1.021. Test Loss: 1.103. Train Acc: 0.451. Test Acc:0.3939
Epoch: 120. Train Loss: 1.002. Test Loss: 1.102. Train Acc: 0.6078. Test Acc:0.3939
Epoch: 130. Train Loss: 0.9812. Test Loss: 1.101. Train Acc: 0.6667. Test Acc:0.4242
Epo

Epoch: 1000. Train Loss: 0.2342. Test Loss: 1.817. Train Acc: 0.9412. Test Acc:0.3939
LangIdRNN2 using retrained-wav2vec_features-c on fold#0
---------------------------------------------------------------------------------
      Layer (type)                  Output Shape         Param #     Tr. Param #
         Dropout-1               [10, 512, 2998]               0               0
          Conv1d-2                 [10, 1, 2996]           1,537           1,537
         Dropout-3                 [10, 1, 2996]               0               0
       MaxPool1d-4                 [10, 1, 1498]               0               0
             GRU-5     [10, 1498, 3], [1, 10, 3]              54              54
          Linear-6                       [10, 3]              12              12
Total params: 1,603
Trainable params: 1,603
Non-trainable params: 0
---------------------------------------------------------------------------------
Epoch: 10. Train Loss: 1.145. Test Loss: 1.132. Train Acc: 

Epoch: 860. Train Loss: 0.1115. Test Loss: 1.47. Train Acc: 1.0. Test Acc:0.4848
Epoch: 870. Train Loss: 0.1104. Test Loss: 1.471. Train Acc: 1.0. Test Acc:0.4848
Epoch: 880. Train Loss: 0.1223. Test Loss: 1.494. Train Acc: 1.0. Test Acc:0.4545
Epoch: 890. Train Loss: 0.1301. Test Loss: 1.475. Train Acc: 1.0. Test Acc:0.4848
Epoch: 900. Train Loss: 0.1207. Test Loss: 1.473. Train Acc: 1.0. Test Acc:0.4848
Epoch: 910. Train Loss: 0.09822. Test Loss: 1.476. Train Acc: 1.0. Test Acc:0.4848
Epoch: 920. Train Loss: 0.1065. Test Loss: 1.478. Train Acc: 1.0. Test Acc:0.4848
Epoch: 930. Train Loss: 0.1453. Test Loss: 1.532. Train Acc: 0.9804. Test Acc:0.4545
Epoch: 940. Train Loss: 0.1328. Test Loss: 1.521. Train Acc: 0.9608. Test Acc:0.4545
Epoch: 950. Train Loss: 0.09977. Test Loss: 1.498. Train Acc: 1.0. Test Acc:0.4848
Epoch: 960. Train Loss: 0.1416. Test Loss: 1.502. Train Acc: 0.9804. Test Acc:0.4848
Epoch: 970. Train Loss: 0.08972. Test Loss: 1.599. Train Acc: 1.0. Test Acc:0.5152
Epoch

Epoch: 720. Train Loss: 0.2042. Test Loss: 1.505. Train Acc: 0.9804. Test Acc:0.4545
Epoch: 730. Train Loss: 0.18. Test Loss: 1.545. Train Acc: 1.0. Test Acc:0.4242
Epoch: 740. Train Loss: 0.2095. Test Loss: 1.54. Train Acc: 0.9608. Test Acc:0.4242
Epoch: 750. Train Loss: 0.1705. Test Loss: 1.687. Train Acc: 0.9804. Test Acc:0.3636
Epoch: 760. Train Loss: 0.1383. Test Loss: 1.558. Train Acc: 1.0. Test Acc:0.4545
Epoch: 770. Train Loss: 0.2181. Test Loss: 1.694. Train Acc: 0.9608. Test Acc:0.3636
Epoch: 780. Train Loss: 0.1527. Test Loss: 1.567. Train Acc: 1.0. Test Acc:0.4545
Epoch: 790. Train Loss: 0.1515. Test Loss: 1.72. Train Acc: 0.9804. Test Acc:0.3939
Epoch: 800. Train Loss: 0.1692. Test Loss: 1.645. Train Acc: 0.9608. Test Acc:0.3939
Epoch: 810. Train Loss: 0.1539. Test Loss: 1.668. Train Acc: 0.9804. Test Acc:0.3939
Epoch: 820. Train Loss: 0.1274. Test Loss: 1.761. Train Acc: 1.0. Test Acc:0.3636
Epoch: 830. Train Loss: 0.1522. Test Loss: 1.778. Train Acc: 0.9804. Test Acc:0.3

KeyboardInterrupt: 